# Solution Model 2
### SSD MobileNet V2 FPNLite 640x640

In [ ]:
!pip install tensorflow_io sagemaker -U

In [ ]:
import os
import sagemaker
from framework import CustomFramework

In [ ]:
role = sagemaker.get_execution_role()
print(role)

In [ ]:
inputs = {
    "train": "s3://cd2688-object-detection-tf2/train/",
    "val": "s3://cd2688-object-detection-tf2/val/"
}

tensorboard_s3_prefix = "s3://object-detection-project-singhsegv/ssd-mobilenet-v2-fpnlite/logs/"

In [ ]:
%%bash

git clone https://github.com/tensorflow/models.git docker/models

cp docker/models/research/object_detection/exporter_main_v2.py source_dir
cp docker/models/research/object_detection/model_main_tf2.py source_dir

In [ ]:
image_name = "tf2-object-detection-ssd-mobilenet-v2-fpnlite"
!sh ./docker/build_and_push.sh $image_name

In [ ]:
with open(os.path.join("docker", "ecr_image_fullname.txt"), "r") as f:
    container = f.readlines()[0][:-1]

print(container)

In [ ]:
container = "481217455434.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection-faster-rcnn-resnet152:20240129073701"

In [ ]:
%%bash

mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/ssd_mobilenet_v2_fpnlite.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssd_mobilenet_v2_fpnlite.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint


In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path="/opt/training/"
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point="run_training.sh",
    source_dir="source_dir",
    hyperparameters={
        "model_dir": "/opt/training",
        "pipeline_config_path": "ssd_mobilenet_v2_fpnlite.config",
        "num_train_steps": "2000",
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type="ml.g5.xlarge",
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name="tf2-object-detection-ssd-mobilenet-v2-fpnlite"
)

estimator.fit(inputs)